In [1]:
import numpy as np
import math

In [96]:
def cannonization(matrix_arr, z_arr, B_arr,signs, max_min):
    B = np.copy(B_arr)
    z = np.copy(z_arr)
    matrix = np.copy(matrix_arr)
    
    def switch_case(matr,sings):
        new_matrix = []
        n = matr.shape[0]
        k = matr.shape[1]
        flag = True
        app_els = 0
        
        for sign in signs:
            if sign != '=':
                app_els +=1        
        Eq = []
        Bigger = []
        Smaller = []

        flag = True
        new_matrix = matr.tolist()
        for i in range(n):
            for j in range(app_els):
                new_matrix[i].append(0)
        
        counter = 0
        for i in range(n):
            if signs[i] == '=':
                counter +=1
            elif signs[i] == '<=' or '<':
                new_matrix[i][k+counter-1] = 1
                counter +=1
            elif sign == '>=' or '>':
                for el in new_matrix[i]:
                    el *= -1
                new_matrix[i][k+counter-1] = 1
                
        '''for i in range(n):
            if Smaller[i] >0:
                new_matrix[[n]
            else:
                if not flag:
                    new_matrix[i].append(0)
                
            
        

        for i in range(len(sings)):
            value = signs[i]
            if value == '=':
                new_matrix.append(matr[i].tolist())
                if app_els >0:
                    for i in range(app_els):
                        new_matrix[-1].append(0)
            elif value == '<=' or value == '<':
                new_els = [0]*(i-1)
                new_els.append(1)
                new_matrix.append(np.append(matr[i],new_els).tolist())
                flag = False
            elif value == '>=' or value == '>':
                matr[i] *= -1
                new_els = [0]*(i-1)
                new_els.append(1)
                new_matrix.append(np.append(matr[i],new_els).tolist())
                flag = False
            else:
                return Exception('Write correct signs!')'''
        
        return new_matrix
    matrix = switch_case(matrix,signs)
    if max_min == 'min':
        matrix *= -1
        B *= -1
    '''for i in range(B.shape[0]):
        matrix[i].append(B[i])'''
    return matrix

In [173]:
def give_basis(matrix,z,B,m):
    #m = 10000
    n = matrix.shape[0]
    basis_indices = [None]*(n)
    for col_idx in range(matrix.shape[1]):
        col = matrix[:,col_idx]
        non_zero_count = np.count_nonzero(col)
        if non_zero_count == 1:
            row_index = np.argmax(col)
            basis_indices[row_index] = col_idx
            
    for i in range(n):
        if basis_indices[i] is None:
            column = [0]*(n)
            column[i] = 1
            matrix = np.column_stack((matrix,column))
            basis_indices[i] = matrix.shape[1]-1
            z = np.append(z,-m)
    print(z)
    return matrix, basis_indices, z
            
            
            
def processing_data(restr,z,B_arr,signs, max_min,m):
    matrix_old = np.array(restr)
    B = np.array(B_arr)
    n = matrix_old.shape[0]
    z = np.array(z)
    #step1
    matrix = cannonization(matrix_old,z,B_arr,signs,max_min)
    new_signs = ['=' for i in range(n)]
    #step 2
    for i in range(n):
        if B[i] <0:
            matrix[i] *= -1
            B[i] *= -1
    #step3
    matrix, basis,z = give_basis(matrix_old,z,B,m)
    for i in range(n):
        print(matrix[i][basis[i]])
    cols = matrix.shape[1] - z.shape[0]
    columns_z = [0]*cols
    z = np.concatenate((z,columns_z))
    #new_z = z.reshape(1,-1)
    #matrix = np.concatenate((matrix,new_z), axis = 0)
    #B = np.append(B,0)
    matrix = np.column_stack((matrix,B))
    print(matrix)
    return matrix, basis, z

In [194]:
def default_status():
    raise Exception('Некорректный ввод!')


def status_option(action):
    def max_status():
        return True
    def min_status():
        return False
    
    actions = {
        'max': max_status,
        'min': min_status
    }

    return actions.get(action,default_status)()

def recount(matrix_input, Z_resh, str_resh):
    
    matrix = np.copy(matrix_input)
    matrix = matrix.astype(np.float64)

    matrix[str_resh] /= matrix[str_resh][Z_resh]

    for i in range(matrix.shape[0]):
        if i != str_resh:
            
            # Правило прямоугольника
            matrix[i] -= matrix[i][Z_resh] * matrix[str_resh]
    print(np.around(matrix, decimals = 2))
    return matrix

def get_new_basis(matrix_input, col_index):
        matr = np.copy(matrix_input)
        p0 = matr[:,matr.shape[1]-1]
        pi = matr[:,col_index]
        max_el = np.amax(pi)
        pi[pi == 0] = -1
        teta = p0 / pi
        teta = np.where(teta > 0, teta, np.inf)
        resh_v = teta.argmin()
        if teta[resh_v] == np.inf:
                raise Exception("No solutions")
        return resh_v

def get_new_z(matrix, z_old,col_index,resh_v,max_min):
    new_z = np.copy(z_old)
    new_z[col_index] = 0.0
    for i in range(new_z.shape[0]):
        if i != col_index:
                new_z[i] -= (matrix[resh_v][i]*(z_old[col_index])) / matrix[resh_v][col_index]
    if status_option(max_min):
            if new_z[new_z.shape[0]-1] < z_old[z_old.shape[0]-1]:
                raise Exception("Not solution :(")
    else:
        if new_z[new_z.shape[0]-1] > z_old[z_old.shape[0]-1]:
                    raise Exception("Not solution :(")
    return new_z

def symplex_method(matrix,z,old_basis, max_min):
    matr = matrix.copy()
    basis = old_basis.copy()
    n = matrix.shape[0]
    k = z.shape[0]
    if 2>1:
        # максимальный элемент в z строке
        if status_option(max_min):
            col_index = np.argmin(z[0:k-1])
        else:
            col_index = np.argmax(z[0:k-1])
        
        # находим новый базис
        resh_v = get_new_basis(matrix,col_index)
        basis[resh_v] = col_index

        
            
        new_z = get_new_z(matrix,z,col_index,resh_v,max_min)
        #z -= max_el*matrix[resh_v].astype(np.float64)
        
        matrix = recount(matrix,col_index,resh_v)
        print("Базис", basis)
        print("Решающий столбец: ", col_index)
        print("Решающая строка: ", resh_v)
        print("Решающий элемент: ", matrix[resh_v][col_index])
        print(np.around(matrix,decimals = 2))
        print("------------------------------------")
        print(new_z)
        
    
    '''if status_option(max_min):
        new_z = -new_z'''
    return matrix, new_z, basis

def continue_solve(mark_in, max_min):  # проверка положительных оценок
    mark = np.copy(mark_in)
    mark = mark[1:]
    if status_option(max_min):
        for i in mark:
            if i < 0:
                return True,
        return False
    for i in mark:
            if i > 0:
                return True,
    return False

def symplex_sol(matrix_old, z, B_arr,signs, max_min,m):
    flag = True
    iter = 0
    x_matrix = np.array(matrix_old)
    z_matrix = np.array(z)
    
    x_matrix, basis, z_matrix = processing_data(x_matrix,z, B_arr,signs,max_min,m)
    
    z_matrix = z_matrix.astype(np.float64)
    z_matrix = np.append(z_matrix,0.0)
    for i in range(x_matrix.shape[0]):
        for k in range(z_matrix.shape[0]):
                z_matrix[k] += x_matrix[i][k]*m
                
    print("start basis:  ", basis)
    print('start matrix: ', x_matrix)
    print('start_z_line: ', z_matrix)
    
    while flag:
        print(f"matrix: {matrix}")
        print(f"z_matrix {z_matrix}")
        x_matrix, z_matrix, basis = symplex_method(x_matrix,z_matrix,basis,'max')
        flag = continue_solve(z_matrix,max_min)
        iter +=1
        if iter > 20:
            break
    return x_matrix, z_matrix, basis

In [195]:
def main():
    z = [8,-2,4,-7,4]
    restr = [[4,-7,6,5,-2],
         [9,2,-9,3,9],
         [-1,5,2,-4,5]]
    signs = ['<=','<=', '<=']
    B = np.array([9,6,6])
    restr = np.array(restr)
    z = np.array(z)
    matrix, z_res, basis = symplex_sol(restr, z, B ,signs, 'max',-1000)
    ans = z_res[-1]
    x = [0]*(z_res.shape[0] - (matrix.shape[0]+1))
    print("basis", basis)
    for i in range(len(basis)):
        x[basis[i]] = matrix[i][-1]
    print(f"z= {ans} with x: {x}")

In [196]:
main()

[   8   -2    4   -7    4 1000 1000 1000]
1
1
1
[[ 4 -7  6  5 -2  1  0  0  9]
 [ 9  2 -9  3  9  0  1  0  6]
 [-1  5  2 -4  5  0  0  1  6]]
start basis:   [5, 6, 7]
start matrix:  [[ 4 -7  6  5 -2  1  0  0  9]
 [ 9  2 -9  3  9  0  1  0  6]
 [-1  5  2 -4  5  0  0  1  6]]
start_z_line:  [-1.1992e+04 -2.0000e+00  1.0040e+03 -4.0070e+03 -1.1996e+04  0.0000e+00
  0.0000e+00  0.0000e+00 -2.1000e+04]
matrix: [[ 3 -2  9  9 -7  0  0  1]
 [ 8  3 -3  0  3  1  0  2]
 [ 1  6  3  6 -5  0  1  9]]
z_matrix [-1.1992e+04 -2.0000e+00  1.0040e+03 -4.0070e+03 -1.1996e+04  0.0000e+00
  0.0000e+00  0.0000e+00 -2.1000e+04]
[[ 6.   -6.56  4.    5.67  0.    1.    0.22  0.   10.33]
 [ 1.    0.22 -1.    0.33  1.    0.    0.11  0.    0.67]
 [-6.    3.89  7.   -5.67  0.    0.   -0.56  1.    2.67]]
Базис [5, 4, 7]
Решающий столбец:  4
Решающая строка:  1
Решающий элемент:  1.0
[[ 6.   -6.56  4.    5.67  0.    1.    0.22  0.   10.33]
 [ 1.    0.22 -1.    0.33  1.    0.    0.11  0.    0.67]
 [-6.    3.89  7.   -5.67  0

In [182]:
z = [0,-2,-2,1,4]
restr = [[3,-2,9,9,-7],
         [8,3,-3,0,3],
         [1,6,3,6,-5,]]
B = [1,2,9]
restr = np.array(restr)
z = np.array(z)
signs = ['=','<=', '>']
res = cannonization(restr,z,B,signs,'max')
print(res)
matrix = np.array(res)
matrix_res, z_res, basis = symplex_sol(restr,z,B,signs,'max',1000)

[[3, -2, 9, 9, -7, 0, 0, 1], [8, 3, -3, 0, 3, 1, 0, 2], [1, 6, 3, 6, -5, 0, 1, 9]]
[    0    -2    -2     1     4 -1000 -1000 -1000]
1
1
1
[[ 3 -2  9  9 -7  1  0  0  1]
 [ 8  3 -3  0  3  0  1  0  2]
 [ 1  6  3  6 -5  0  0  1  9]]
start basis:   [5, 6, 7]
start matrix:  [[ 3 -2  9  9 -7  1  0  0  1]
 [ 8  3 -3  0  3  0  1  0  2]
 [ 1  6  3  6 -5  0  0  1  9]]
start_z_line:  [12000.  6998.  8998. 15001. -8996.     0.     0.     0.]
matrix: [[ 3 -2  9  9 -7  0  0  1]
 [ 8  3 -3  0  3  1  0  2]
 [ 1  6  3  6 -5  0  1  9]]
z_matrix [12000.  6998.  8998. 15001. -8996.     0.     0.     0.]
[[21.67  5.    2.    9.    0.    1.    2.33  0.    5.67]
 [ 2.67  1.   -1.    0.    1.    0.    0.33  0.    0.67]
 [14.33 11.   -2.    6.    0.    0.    1.67  1.   12.33]]
Базис [5, 4, 7]
Решающий столбец:  4
Решающая строка:  1
Решающий элемент:  1.0
[[21.67  5.    2.    9.    0.    1.    2.33  0.    5.67]
 [ 2.67  1.   -1.    0.    1.    0.    0.33  0.    0.67]
 [14.33 11.   -2.    6.    0.    0.    1.67

In [132]:
z = [0,-2,-2,1,4]
restr = [[3,-2,9,9,-7],
         [8,3,-3,0,3],
         [1,6,3,6,-5,]]
B = [1,2,9]
restr = np.array(restr)
z = np.array(z)
signs = ['=','<=', '>']
matrix = processing_data(restr,z,B,signs,'max')
matrix_res, z_res, basis = symplex_sol()

[5, 6, 7]
[[    3    -2     9     9    -7 10000     0     0]
 [    8     3    -3     0     3     0 10000     0]
 [    1     6     3     6    -5     0     0 10000]]
10000
10000
10000
3
[[ 0 -2 -2  1  4  0  0  0]]
[[    3    -2     9     9    -7 10000     0     0     1]
 [    8     3    -3     0     3     0 10000     0     2]
 [    1     6     3     6    -5     0     0 10000     9]
 [    0    -2    -2     1     4     0     0     0     0]]


In [48]:
import numpy as np

arr = np.array([1, 2, 3])
new_elements = [4, 5, 6]
new_arr = np.concatenate((arr, new_elements))
print(new_arr)

[1 2 3 4 5 6]


In [51]:
new_arr *= -1
print(new_arr)
print(new_arr.shape[1])

[-1 -2 -3 -4 -5 -6]


IndexError: tuple index out of range

In [91]:
arr = np.array([[1,2],
       [2,3]])
k = arr.shape[1]
print(arr[0][k-1])

2


In [98]:
import numpy as np

matrix = np.array([[3, -2, 9, 9, -7, 0, 0, 1],
                  [8, 3, -3, 0, 3, 1, 0, 2],
                  [1, 6, 3, 6, -5, 0, 1, 9]])

z = np.array([0, -2, -2, 1, 4])

# Итерируемся по столбцам матрицы и проверяем, есть ли только одна ненулевая ячейка
basis_indices = []
for col_index in range(matrix.shape[1]):
    col = matrix[:, col_index]
    non_zero_count = np.count_nonzero(col)
    if non_zero_count == 1:
        row_index = np.argmax(col)
        basis_indices.append((row_index, col_index))

# Выводим индексы базисных переменных
print("Индексы базисных переменных:")
for row_index, col_index in basis_indices:
    print(f"Строка {row_index}, Столбец {col_index}")

# Создаем матрицу базиса и вектор z базисных переменных
basis_matrix = matrix[:, [col_index for _, col_index in basis_indices]]
z_basis = z[[col_index for _, col_index in basis_indices]]

print("Матрица базиса:")
print(basis_matrix)

print("Вектор z базисных переменных:")
print(z_basis)


Индексы базисных переменных:
Строка 1, Столбец 5
Строка 2, Столбец 6


IndexError: index 5 is out of bounds for axis 0 with size 5

In [101]:
import numpy as np

# Исходный массив
arr = np.array([[1, 2],
                [3, 4]])

# Столбец для добавления
new_column = np.array([5, 6])

# Добавление столбца
new_arr = np.column_stack((arr, new_column))
new_arr

array([[1, 2, 5],
       [3, 4, 6]])